# Scipp widgets introduction

[Scipp](https://scipp.github.io/) is a python data structure module for handling multi-dimensional data arrays with labeled dimensions. 
Data reduction or analysis workflows using this module often take the form of ipython notebooks. 


Scipp-widgets is a separate module containing some simple scipp specific wrappers using [ipywidgets](https://ipywidgets.readthedocs.io/) to allows graphical elements to be easily incorporated into such notebooks.
This can be installed using conda with the command `conda install -c scipp/label/dev scipp-widgets`.

In this context, a widget is a generic term describing the building blocks from which a user interface can be constructed.
As widgets are often nested, this can lead to the entire interface being a widget that contains sub-widgets within it.

## Introductory examples
For demonstration purposes we first create a simple set of [scipp](https://scipp.github.io/tutorials/multi-d-datasets.html) data.

In [1]:
import numpy as np
import scipp as sc
var =sc.Variable(['x', 'z'], values=np.arange(0.0, 10.0, 0.1).reshape(10, 10),
                                              variances=0.1*np.random.rand(10, 10))
sc.show(var)

## Wrapping a processing function
Scipp-widgets allows functions to be wrapped in simple graphical interfaces.
Taking inputs from, and returning outputs to, the notebook's global scope.
As a first example, take the `scipp.sum` function.

We will need to create an input specification object describing how we want the graphical inputs displayed, what validation we wish to perform, and the argument of the underlying function to which they should correspond.
Currently there is no way to specify layout options.

In [2]:
from scipp_widgets import Input, TextInput
data_input = Input('x', placeholder='input data')
dimension_input = TextInput('dim', options = ('x', 'z'))

A simple user interface which wraps the sum function can then be constructed like so:

In [12]:
from scipp_widgets import ProcessWidget
ProcessWidget(sc.sum, [data_input, dimension_input, data_input, data_input, data_input, data_input], layout='row wrap')

ProcessWidget(children=(Box(children=(Combobox(value='', placeholder='scipp_obj'), Combobox(value='', options=…

## Wrapping a display function

Another common case is to wrap a function and display its result without adding it to the notebook's global scope.
As an example, take the `scipp.plot.plot` function. This takes a scipp object and plots it. 

In [4]:
from scipp_widgets import DisplayWidget
data_input = Input('scipp_obj')
DisplayWidget(sc.plot.plot, [data_input])

DisplayWidget(children=(Box(children=(Combobox(value='', placeholder='scipp_obj'), Button(description='plot', …

As plotting is a very common operation there is also a seperate `PlotWidget` object which acts as a shothand for the above.

In [5]:
from scipp_widgets import PlotWidget
PlotWidget()

DisplayWidget(children=(Box(children=(Combobox(value='', placeholder='scipp_obj'), Button(description='plot', …

## Validators
Validator methods can be added to any input. 
These are run when the widget is processing and serve to both perform any pre-processing the input needs before it is passed to the underlying function, as well as perform any validation specified.

For example, we could add a validator to our wrapped plot function meaning it will only accept scipp objects as inputs.

In [6]:
from scipp_widgets.validators import ScippObjectValidator
scipp_object_validator = ScippObjectValidator()
data_input = Input('scipp_obj', validator=scipp_object_validator, tooltip='Data to plot')
DisplayWidget(sc.plot.plot, [data_input])

DisplayWidget(children=(Box(children=(Combobox(value='', placeholder='scipp_obj'), Button(description='plot', …

## Hiding code
The code block used to create a widget can be hidden using the key word argument `hide_code`.
This adds an additional button labelled `Py` to toggle the visibility of the preceding code block.

In [7]:
data_input = Input('scipp_obj', validator=scipp_object_validator, tooltip='Data to plot')
DisplayWidget(sc.plot.plot, [data_input], hide_code=True)

<IPython.core.display.Javascript object>

DisplayWidget(children=(Box(children=(Combobox(value='', placeholder='scipp_obj'), Button(description='plot', …

## Further reading
For a full list of the available widgets, inputs and validators see:

* [widgets list](widgets.ipynb)

* [inputs list](inputs.ipynb)

* [validators list](validators.ipynb)
 
For information on how to write your own inputs and validators see [concepts](concepts.ipynb)